In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection, utils

In [ ]:
from ipynb.fs.full.EdfManipulation import read_and_store_data
from ipynb.fs.full.FeatureSelection import dimentionalityReduction
from ipynb.fs.full.SplitDataset import createTrainingAndTestDatasets
from ipynb.fs.full.DatasetBalancing import minorityOversampling, majorityUndersampling
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes
from ipynb.fs.full.FeatureNormalization import featureNormalization, removeNonNumericValues
from ipynb.fs.full.FeatureClassificationMethods import CompleteSVM, CompleteKNN, CompleteNB, CompleteDT, CompleteRF, CompleteLDA, CompleteLR, CompleteLSTM
from ipynb.fs.full.FeatureComputation import leftRightHemisphericChannels, featureExtractionLeftRight, averageChannels, featureExtractionAverage, featureExtractionFull

### Feature Extraction

In [ ]:
def featureExtraction (df, sample_rate, step, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, experiment):
    if experiment.upper() == 'FULL':
        ft = pd.DataFrame(featureExtractionFull (df, sample_rate, step))
    elif experiment.upper() == 'AVERAGE':
        ft = pd.DataFrame(featureExtractionAverage (averageChannels(df), sample_rate, step))
    else:
        ft = pd.DataFrame(featureExtractionLeftRight (leftRightHemisphericChannels(df), sample_rate, step))
    removeNonNumericValues(ft)
    ft = featureNormalization(ft)
    print('Normalized features')
    removeNonNumericValues(ft)
    ft = dimentionalityReduction(ft, pca_tolerance)
    removeNonNumericValues(ft)
    ft = majorityUndersampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], undersampling_rate, undersampling_neighbors, undersampling_method)
    removeNonNumericValues(ft)
    ft = utils.shuffle(minorityOversampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], oversampling_neighbors, oversampling_method))
    ft.reset_index(drop = True, inplace = True)
    removeNonNumericValues(ft)
    return ft

### Create Training and Testing Data

In [ ]:
def TrainTestData (features, test_ratio, k_fold, perfInd):
    train_dat, test_dat, train_ind, test_ind = createTrainingAndTestDatasets(features, test_ratio)
    results = pd.DataFrame(columns = perfInd)
    kf = model_selection.KFold(n_splits = k_fold, shuffle = True)
    return train_dat, test_dat, train_ind, test_ind, results, kf

### Experiment

In [ ]:
def Experiment (df, channels, dataset, sample_rate, time_window, test_ratio, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, k_fold, knn_neighbors, rf_estimators, max_iter, dropout, loss_function, epochs, lstm_units, dense1_units, dense2_units, batch, step, perfInd, experiment):
    if  experiment.upper() not in ['FULL', 'AVERAGE', 'LEFTRIGHT']:
        print('No such experiment:', experiment)
        return
    else:
        print ('Executing Experiment', experiment)
    ft = featureExtraction (df, sample_rate, step, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, experiment)
    train_dat, test_dat, train_ind, test_ind, results, kf = TrainTestData (ft, test_ratio, k_fold, perfInd)
    CompleteSVM(train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd)
    CompleteKNN(train_dat, test_dat, train_ind, test_ind, results, knn_neighbors, ft, kf, perfInd)
    CompleteNB(train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd)
    CompleteDT(train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd)
    CompleteRF(train_dat, test_dat, train_ind, test_ind, results, rf_estimators, ft, kf, perfInd)
    CompleteLDA(train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd)
    CompleteLR(train_dat, test_dat, train_ind, test_ind, results, max_iter, ft, kf, perfInd)
    CompleteLSTM(train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd, lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function, batch)
    return results